# 2020_04_24
### Plan for the day: 
1. Make a script that will create all nessasery files in the a project folder.
 - test if .gitignore works 
2. Transfer the files needed for the analysis 
3. Make sript for aligment of the files 
  - test how long it takes 

### 1. Script that will create structure of the directories 
Complited: **yes**

Working: **yes**

Saved as a separate R file: **yes**

In [1]:
#This sript will create all nessesary folder for a tiypical R project. 
#Also script will create a file for github to ignor all files but scr folder 
#Prior runing the script set working directory (setwd()) to your R project folder. 
#The script will add only not extisting directories 
####################################################

#1. Add directories 
l.dir <- c("scr", "data", "output", "resourses", 
           "scr/functions", 
           "data/raw", "data/metadata", 
           "output/plots", "output/objects")

for (d in l.dir) {
    dir.create(d, showWarnings = FALSE)
}

#2. Add .gitignore file with. 
#   All folders but scr are ignored by defult 
fileConn<-file("gitignore.txt")
writeLines(c("data", "output", "resourses", 
           "data/raw", "data/metadata", 
           "output/plots", "output/objects", 
            ".ipynb_checkpoints"), fileConn)
close(fileConn)

file.rename("gitignore.txt", ".gitignore")

[1] TRUE

### 2. Script to make aligment of the sequneces from the first database and second 
Complited: 

Working: 

Saved as separate R file: 

#### 2.1 load.abs.install.pkg function
Complited: **yes**

Type: **Generlalized**

Saved: **no**

In [1]:
#The following function (load.abs.install.pkg) will load specified packages and install any missing form the list.
#BiocManager is used for installation

load.abs.install.pkg <- function(list.of.packages){
    
    if (!requireNamespace("BiocManager", quietly = TRUE))    
        install.packages("BiocManager")
    
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]

    if(length(new.packages)) BiocManager::install(new.packages)    
    
    lapply(list.of.packages, require, character.only = TRUE)
}


**Load required packages**

In [2]:
#load (install if missing) libararies using defined above function load.abs.install.pkg
# foreach and doParallel for parrallel computing 
# msa for alignment 
load.abs.install.pkg(c("msa", "foreach", "doParallel", "phyloseq", "parallel", "tidyverse", "Biostrings"))

Loading required package: msa
Loading required package: Biostrings
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors
Loading req

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

**Load data**

The data will be used from privious runs. It will be copied manually into resourses folder. 

Phyloseq objects will be used as input 

In [3]:
# Read in phyloseq from the first run (filtered)
psI.1.1 <- readRDS("resourses/ps1.1.rds")

#Read in phyloseq from the second run (unfiltered)
psII.0 <- readRDS("resourses/phyloseq0.rds")

**Extract seqnces from phyloseques**
- sequences from psI will be use as refferece

In [4]:
ps.I.tax <- psI.1.1@tax_table@.Data

seq.I <- rownames(ps.I.tax)

seq.II.all <- colnames(psII.0@otu_table@.Data)



### Align sequences 

Sequences from the second sequencesing run will be align against sequences from the first sequencing run that were used to build an rf model. 

#### 2.1 paired.aling function

Complited: **no**

Type: **project_specific**

Dependeses: **yes** *Biostrings, foreach, doParallel*

Saved: **no**

In [ ]:
aling.against.list <- function(ref.seq.list, target.seq, subsMatrix) {
    score.m <- c()
    best.match.score = -10^6
    best.match.n = 0
    for (seqA in 1:length(ref.seq.list)) {
        align.s <- pairwiseAlignment(ref.seq.list[seqA], target.seq, substitutionMatrix = subsMatrix, 
                               gapOpening = - 1, gapExtension = - 1 , scoreOnly = TRUE)
        if (best.match.score < align.s) {
            best.match.score <- align.s
            best.match.n <- seqA}
        }
    best.match <- c(target.seq, ref.seq.list[best.match.n], best.match.n, best.match.score)
    names(best.match) <- c("TargetSeq", "BestMatchedRefSeq", "BestMatchedRefSeqNumber", "Score")
    return(best.match)
    }

In [68]:
# Make sabstitution matrix for pairwiseAlignment
sub.mat<- nucleotideSubstitutionMatrix(match = 0, mismatch = 1, baseOnly = TRUE)

##### Execution of function above in parallel using foreach 

In [88]:
#Register the clustor 
cl <- makeCluster(48)
registerDoParallel(cl)

In [89]:
#foreach loop
ts3 <- Sys.time()
best.hit.m <- foreach (i=1:length(seq.II.all), 
                       .combine="rbind", 
                       .packages = "Biostrings") %dopar% {aling.against.list(ref.seq.list = seq.I, 
                                                                             target.seq = seq.II.all[i], 
                                                                             subsMatrix = sab.mat)}
te3 <- Sys.time()
te3 - ts3

Time difference of 47.85176 mins

In [90]:
stopCluster(cl)

In [94]:
#Convert to dataframe and save 
best.hit.m <- data.frame(test80each)
write_csv(best.hit.m, "output/objects/best_hit_m.csv")

##### Visualisation of differences 


In [102]:
best.hit.m$Score <- as.numeric(as.character(best.hit.m$Score))
hist.score <- ggplot(best.hit.m, aes(x=Score)) + 
                    geom_histogram(binwidth=0.5) + 
                    theme_bw()
ggsave(hist.score, filename = "output/plots/histogram_score.pdf", width = 6.5, height = 3)
ggsave(hist.score, filename = "output/plots/histogram_score.png", dpi=300, width = 6.5, height = 3)

##### Add taxonomic information for sequences 

In [126]:
#prepare taxonomic data 
ps.I.tax[is.na(ps.I.tax)] <- "Unknown"
ps.II.tax <- psII.0@tax_table@.Data
ps.II.tax[is.na(ps.II.tax)] <- "Unknown"

In [127]:
tax.ref <- c("Sequence", "Taxonomy")
for (i in best.hit.m$BestMatchedRefSeq) {
     tax.ref <- rbind(tax.ref, c(i, paste(ps.I.tax[rownames(ps.I.tax) %in% i, ], collapse="__")))
}

In [129]:
tax.target <- c("Sequence", "Taxonomy")
for(i in best.hit.m$TargetSeq) {
    tax.target <- rbind(tax.target, c(i, paste(ps.II.tax[rownames(ps.II.tax) %in% i, ], collapse="__")))
}